In [1]:
import CodaPCA
import CodaRegressmb as CodaRegress
import numpy as np
from runpca import read_csv
import os
import sklearn
import torch
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
#change module for newer sklearn versions
from sklearn.model_selection  import cross_val_score
from sklearn.model_selection  import KFold
import matplotlib.pyplot as plt
import pandas as pd

W0716 11:45:27.166184 140395222361920 deprecation_wrapper.py:119] From /home/sean/Documents/Honours/honours/Code/coda-pca-orig/coda/codes/CodaPCA.py:14: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0716 11:45:27.167003 140395222361920 deprecation_wrapper.py:119] From /home/sean/Documents/Honours/honours/Code/coda-pca-orig/coda/codes/CodaPCA.py:14: The name tf.logging.ERROR is deprecated. Please use tf.compat.v1.logging.ERROR instead.



In [2]:
#read in the data. Given an array of which files are regression, classification or unlabelled
data_path = os.getcwd() + "/Aitchinson"

regression_list = [3,4,5,18,21,34,39]
classification_list = [7,8,9,11,12,16,17,19,23,24,25,26,28,29,33,37]
unlabelled_list=[1,2,6,10,13,14,15,20,22,27,30,31,32,35,36,38,40]

r_files = []
c_files = []
u_files = []


for file in os.listdir(data_path):
    for i in regression_list:
        if os.path.isfile(os.path.join(data_path,file)) and 'Data ' + str(i) + '.' in file:
            r_files.append("Aitchinson/" + file)
    for i in classification_list:
        if os.path.isfile(os.path.join(data_path,file)) and 'Data ' + str(i) + '.' in file:
            c_files.append("Aitchinson/" + file)
    for i in unlabelled_list:
        if os.path.isfile(os.path.join(data_path,file)) and 'Data ' + str(i) + '.' in file:
            u_files.append("Aitchinson/" + file)

In [3]:
def coda_val(features, targets, n_components, folds):
    targets = targets.reshape(-1,1)
    param_splits = 2
    kfold_scores = []
    for train, test in folds:        
        Y_train = targets[train]
        X_train = features[train]
        
       
        Y_test = targets[test]
        X_test = features[test]
        
        
        kf_inner = KFold(param_splits)
        
        inner_folds = [i for i in kf_inner.split(X_train)]      
            
        #inner loop for parameter selection (lambda term in combined loss):
        param_grid = [0.01,0.05,0.1,0.5,1.0,5.0,10.0,50.0,100.0]
        
        
        for a in param_grid:
            max_error = np.inf
            
   
            
            cval_error = []
            #find the parameter which obtains the best inner cross val score
            for train_inner, test_inner in inner_folds:
                
                model = CodaRegress.CoDA_Regress(features.shape[1], n_components, [10,], [2,])
                
                model.fit(torch.FloatTensor(X_train[train_inner]),  torch.FloatTensor(Y_train[train_inner]), a, lr=1e-2)

                y_inner_pred = model.predict(torch.FloatTensor(X_train[test_inner]))
    
                cval_error.append(sklearn.metrics.mean_squared_error(Y_train[test_inner],y_inner_pred.detach().numpy()))
        
        
                
    
   
            curr_error = np.mean(cval_error)
            if curr_error < max_error:
                max_error = curr_error
                best_param = a
                print ("Current best", best_param)
        
        #compute test score based on best parameter
        best_model = CodaRegress.CoDA_Regress(features.shape[1], n_components, [10,], [2,])
        best_model.fit(torch.FloatTensor(X_train),  torch.FloatTensor(Y_train), a, lr=1e-2)

        y_pred = best_model.predict(torch.FloatTensor(X_test))
        kfold_scores.append(sklearn.metrics.mean_squared_error(Y_test,y_pred.detach().numpy()))
                
    return kfold_scores

    
    
    

In [4]:

#need to specify where the the targets and features are in the dataset, and whether there are non compositional features

def PCA_Regression(data, co_feature_indices, target_index, 
                   other_feature_indices = [], alg=CodaPCA.Alg.CODAPCA, verbose=True):
    
    #can loop through/optimise this in another way?
    
#     headers = data[1]
#     features = data[0][:,co_feature_indices]
#     targets = data[0][:,target_index]
    
    headers = data
    features = data[:,co_feature_indices]
    targets = data[:,target_index]
    
    #normalise the compositional features. TODO anything extra to deal with non compositional features?
    features = np.array([feat/sum(feat) for feat in features])

    #can be empty
    #extra_features = data[0][:,other_feature_indices]
    
    #TODO double check this
    #features = np.hstack([features, extra_features])
    
    #compute the CoDA-PCA projection 
    #TODO add component number as a hyperparameter to optimise 
    n_components=len(co_feature_indices)-2

    pca = CodaPCA.CodaPCA(n_components,lrate=1e-4,nn_shape=[100,100], alg=alg)
    #TODO: check why this is numerically unstable
    #pca = CodaPCA.NonParametricCodaPCA(n_components)

    pca.fit(features)
    
    Y_coda = pca.transform(features)

    pca_clr = CodaPCA.CLRPCA(n_components)
    pca_clr.fit(features)
    
    Y_clr = pca_clr.transform(features)
    
    
    splits = 4
        
    #split data 
    kf = KFold(splits)
        
    folds = [i for i in kf.split(features)]      
    

    lm = Ridge()
    
    coda_score = enhanced_cross_val(lm,Y_coda, targets, folds)
    clr_score = enhanced_cross_val(lm,Y_clr, targets, folds) 
    naive_score = enhanced_cross_val(lm, features, targets, folds)
    
    regress_score = coda_val(features, targets, n_components, folds)
    


    if verbose:
        print("CoDA-PCA:")
        print(coda_score)
        print("CLR-PCA:")
        print(clr_score)
        print ("Naive regression:")
        print (naive_score)
        print ("CoDA-Regress:")
        print (regress_score)
    

    return coda_score,clr_score,naive_score,regress_score

#training methodology as described in:
#https://papers.nips.cc/paper/3215-learning-with-transformation-invariant-kernels.pdf
def enhanced_cross_val(model, features, targets, folds):
    assert len(features) == len(targets), "Mismatch in length of features and targets"
    
    #define the number of splits and folds uised in the parameter selection
    #stick to smaller splits since we have small datasets
    #splits = 4
    param_splits = 3
    #split data 
    #kf = KFold(splits)
    kfold_scores = []
    
    for train, test in folds:        
        Y_train = targets[train]
        X_train = features[train]
        
       
        Y_test = targets[test]
        X_test = features[test]
        
        #inner loop for parameter selection (regularisation term in Ridge Regression):
        param_grid = [0.01,0.05,0.1,0.5,1.0,5.0,10.0,50.0,100.0]
        for a in param_grid:
            max_score = -np.inf
            lm = Ridge(a)
   
            curr_score = np.mean(cross_val_score(lm, X_train, Y_train,cv=param_splits))
            if curr_score > max_score:
                max_score = curr_score
                best_param = a
        
        #compute test score based on best parameter
        lm = Ridge(best_param)
        lm.fit(X_train, Y_train)
        y_pred = lm.predict(X_test)
        kfold_scores.append(sklearn.metrics.mean_squared_error(Y_test,y_pred))
                
    return kfold_scores




In [5]:

#can automate this if we had assume a certain structure for the indices of features and targets, or an array per dataset 

score_dict = {}

#TODO r_files isn't always consistent

data_21_scores = PCA_Regression(read_csv(r_files[0], normalize=False), co_feature_indices=[0,1,2,3], target_index=4)
score_dict['21'] = data_21_scores

data_21_scores2 = PCA_Regression(read_csv(r_files[0], normalize=False), co_feature_indices=[0,1,2,3], target_index=5)
score_dict['21,2'] = data_21_scores2

data_34_scores = PCA_Regression(read_csv(r_files[1], normalize=False), co_feature_indices=[0,1,2,3], target_index=4) 
score_dict['34'] = data_34_scores

data_3_scores = PCA_Regression(read_csv(r_files[2], normalize=False), co_feature_indices=[0,1,2,3,4], target_index=5) 
score_dict['3'] = data_3_scores

data_39_scores = PCA_Regression(read_csv(r_files[3], normalize=False), co_feature_indices=[0,1,2], target_index=3)
score_dict['39'] = data_39_scores

data_4_scores = PCA_Regression(read_csv(r_files[4], normalize=False), co_feature_indices=[0,1,2,3,4], target_index=5)
score_dict['4'] = data_4_scores

data_4_scores2 = PCA_Regression(read_csv(r_files[4], normalize=False), co_feature_indices=[0,1,2,3,4], target_index=6)
score_dict['4,2'] = data_4_scores2

data_5_scores = PCA_Regression(read_csv(r_files[5], normalize=False), co_feature_indices=[0,1,2], target_index=3)
score_dict['5'] = data_5_scores

data_18_scores = PCA_Regression(read_csv(r_files[6], normalize=False), co_feature_indices=[0,1,2], target_index=3)
score_dict['18'] = data_5_scores


loading Aitchinson/Data 21. Permeabilities of bayesite for 21 mixtures of fibres and bonding pressures..csv...
21 samples 6 features
sparsity: 0.0%


TypeError: tuple indices must be integers or slices, not tuple

In [ ]:
score_dict['5'] = data_5_scores
score_dict['18'] = data_18_scores

In [ ]:
#note: plotly code works fine, but gives a jupyter warning when saving with a rendered table

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


init_notebook_mode(connected=True)


table_rows = []
for key in score_dict.keys():
    mean_scores = list(map(np.mean, score_dict[key]))
    table_rows.append([key,*mean_scores])
    
results = [go.Table(
    header=dict(values=["Dataset","CoDA-PCA", "CLR-PCA", "Naive Regression", "CoDA-Regress"]),
    cells=dict(values=np.array(table_rows).T))]

iplot(results, filename = 'basic_table')    
    


In [ ]:
data_5 = read_csv(r_files[5], normalize=False)

data_5[0]

In [ ]:
data = data_5
features = data[0][:,[0,1,2]]
targets = data[0][:,3]
    
#normalise the compositional features. TODO anything extra to deal with non compositional features?
features = np.array([feat/sum(feat) for feat in features])


model = CodaRegress.CoDA_Regress(features.shape[1], 2, [10,], [2,])
model.fit(torch.FloatTensor(features[:30]),  torch.FloatTensor(targets[:30].reshape(-1,1)), 100, lr=1e-2)


In [ ]:
pred = model.predict(torch.FloatTensor(features))
print (pred)
targets

In [6]:
n = 100

x_1 = np.ceil(np.linspace(0,10,n) + 2*np.random.rand(n))
x_2 = np.ceil(np.clip(0, np.linspace(0,10,n) - 3*np.random.rand(n), 10)[::-1])
x_3 = np.ceil(5* np.ones(n) + 5*np.random.rand(n))

X = np.vstack([x_1, x_2, x_3])
X = [[X[0][i], X[1][i], X[2][i]] for i in range(0,X.shape[1])]
Y = np.linspace(0,50, n) + 10*np.random.rand(n)
Y = Y.reshape(-1,1)
data = np.hstack((X,Y))
data

array([[ 1.        , 10.        ,  7.        ,  6.49315753],
       [ 1.        ,  9.        , 10.        ,  9.92850738],
       [ 1.        ,  9.        , 10.        ,  3.28117527],
       [ 1.        ,  7.        , 10.        ,  5.55998836],
       [ 1.        ,  8.        ,  6.        , 10.52704078],
       [ 1.        ,  9.        , 10.        , 11.45608601],
       [ 3.        ,  9.        ,  8.        , 11.18129382],
       [ 1.        ,  7.        ,  9.        ,  5.82068852],
       [ 2.        ,  9.        ,  7.        , 11.73519852],
       [ 2.        ,  8.        , 10.        , 11.99193745],
       [ 2.        ,  8.        ,  7.        ,  6.08879226],
       [ 3.        ,  9.        ,  6.        , 15.46444438],
       [ 2.        ,  7.        ,  6.        ,  8.0425786 ],
       [ 2.        ,  7.        ,  8.        ,  6.60997232],
       [ 2.        ,  6.        ,  7.        ,  7.17303   ],
       [ 2.        ,  8.        , 10.        , 14.09949921],
       [ 3.        ,  9.

In [9]:
synth_scores = PCA_Regression(data, co_feature_indices=[0,1,2], target_index=3)

[epoch     0] L=  3.8219
epoch 1000, loss -167412800.0
epoch 2000, loss -167413456.0
epoch 3000, loss -167413744.0
epoch 4000, loss -167413952.0
epoch 5000, loss -167414048.0
epoch 6000, loss -167414128.0
epoch 7000, loss -167414128.0
epoch 8000, loss -167414144.0
epoch 9000, loss -167413824.0
epoch 10000, loss -167414144.0
epoch 1000, loss 156.55712890625
epoch 2000, loss 49.288753509521484
epoch 3000, loss 44.14034652709961
epoch 4000, loss 18.136119842529297
epoch 5000, loss 15.124115943908691
epoch 6000, loss 14.37248420715332
epoch 7000, loss 14.054534912109375
epoch 8000, loss 13.885627746582031
epoch 9000, loss 13.78336238861084
epoch 10000, loss 13.722427368164062
Current best 0.01
epoch 1000, loss -837070080.0
epoch 2000, loss -837070464.0
epoch 3000, loss -837070784.0
epoch 4000, loss -836963840.0
epoch 5000, loss -837066880.0
epoch 6000, loss -837070976.0
epoch 7000, loss -837062784.0
epoch 8000, loss -837070976.0
epoch 9000, loss -837059456.0
epoch 10000, loss -837068160.0


epoch 2000, loss 54.58909606933594
epoch 3000, loss 21.111629486083984
epoch 4000, loss 18.425792694091797
epoch 5000, loss 18.10063934326172
epoch 6000, loss 18.050552368164062
epoch 7000, loss 18.042776107788086
epoch 8000, loss 18.038734436035156
epoch 9000, loss 18.031999588012695
epoch 10000, loss 18.019657135009766
Current best 0.1
epoch 1000, loss -8370705920.0
epoch 2000, loss -8370706432.0
epoch 3000, loss -8370707456.0
epoch 4000, loss -8370707456.0
epoch 5000, loss -8370707968.0
epoch 6000, loss -8370707968.0
epoch 7000, loss -8370708480.0
epoch 8000, loss -8370707456.0
epoch 9000, loss -8370707456.0
epoch 10000, loss -8370707968.0
epoch 1000, loss 192.81190490722656
epoch 2000, loss 127.7292251586914
epoch 3000, loss 61.376548767089844
epoch 4000, loss 57.164520263671875
epoch 5000, loss 56.72965621948242
epoch 6000, loss 56.663814544677734
epoch 7000, loss 56.65459442138672
epoch 8000, loss 56.64927291870117
epoch 9000, loss 56.64261245727539
epoch 10000, loss 56.630615234

epoch 1000, loss -75338358784.0
epoch 2000, loss -75338350592.0
epoch 3000, loss -75338350592.0
epoch 4000, loss -75337170944.0
epoch 5000, loss -75338342400.0
epoch 6000, loss -75338096640.0
epoch 7000, loss -75320877056.0
epoch 8000, loss -75337785344.0
epoch 9000, loss -75338358784.0
epoch 10000, loss -75337940992.0
epoch 1000, loss 498.6622619628906
epoch 2000, loss 498.6347351074219
epoch 3000, loss 498.6337890625
epoch 4000, loss 498.63385009765625
epoch 5000, loss 498.7197265625
epoch 6000, loss 498.63385009765625
epoch 7000, loss 498.6338195800781
epoch 8000, loss 498.6338195800781
epoch 9000, loss 498.6339111328125
epoch 10000, loss 498.7012939453125
Current best 5.0
epoch 1000, loss -150676701184.0
epoch 2000, loss -150676783104.0
epoch 3000, loss -150676750336.0
epoch 4000, loss -150676783104.0
epoch 5000, loss -150676783104.0
epoch 6000, loss -150676471808.0
epoch 7000, loss -150674276352.0
epoch 8000, loss -150674538496.0
epoch 9000, loss -150673752064.0
epoch 10000, loss 

epoch 3000, loss 4905.78564453125
epoch 4000, loss 4905.7880859375
epoch 5000, loss 4905.83642578125
epoch 6000, loss 4905.78662109375
epoch 7000, loss 4905.787109375
epoch 8000, loss 4905.7978515625
epoch 9000, loss 4905.78662109375
epoch 10000, loss 4905.78564453125
Current best 50.0
epoch 1000, loss -167377715200.0
epoch 2000, loss -167377715200.0
epoch 3000, loss -167377715200.0
epoch 4000, loss -167377715200.0
epoch 5000, loss -167377715200.0
epoch 6000, loss -167377731584.0
epoch 7000, loss -167377731584.0
epoch 8000, loss -167377731584.0
epoch 9000, loss -167377715200.0
epoch 10000, loss -167377715200.0
epoch 1000, loss 9848.5439453125
epoch 2000, loss 9826.0029296875
epoch 3000, loss 9809.474609375
epoch 4000, loss 9801.712890625
epoch 5000, loss 9801.7080078125
epoch 6000, loss 9801.7109375
epoch 7000, loss 9801.716796875
epoch 8000, loss 9801.71875
epoch 9000, loss 9801.7099609375
epoch 10000, loss 9801.7060546875
Current best 100.0
epoch 1000, loss -167377715200.0
epoch 2000

In [10]:
[np.mean(synth_scores[i]) for i in range(0,4)]

[344.5945490895788, 224.6017065025106, 360.06381366209484, 163.2314378373324]

In [8]:
[np.mean(synth_scores[i]) for i in range(0,4)]

[170.76430143923963, 224.6017065025106, 360.06381366209484, 189.66704073881988]

In [ ]:
test = [1,2,3,4,5,6,7]

np.random.shuffle(data)

data
